In [42]:
import pandas as pd
import pinecone
from ast import literal_eval
from sentence_transformers import SentenceTransformer, util
import requests
import os
from pinecone import ServerlessSpec
import time



In [193]:
try:
  import yaml
  with open('config.yaml') as f:
      data = yaml.load(f, Loader=yaml.FullLoader)
      api_hugging = data['API_HUGGING']
      pinecone_api_key = data['API_PINECONE']
except Exception as e:
  print(e)

In [34]:
def concatenar_lista(lista):
    lista = literal_eval(lista)
    return ', '.join(lista)

def string_to_list(lista):
    lista = literal_eval(lista)
    return lista

In [35]:
df = pd.read_csv('25k IMDb movie Dataset.csv')
df = df.fillna(' ')
df['Keywords'] = df['Plot Kyeword'].apply(concatenar_lista)
df['Stars'] = df['Top 5 Casts'].apply(concatenar_lista)
df['Generes'] = df['Generes'].apply(string_to_list)
df['Rating'] = pd.to_numeric(df['Rating'], errors="coerce").fillna(0).astype("float")
df

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path,Keywords,Stars
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...
3,Lightyear,"$71,101,257",5.2,32K,"[Animation, Action, Adventure]",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/,galaxy spaceship robot rocket space adventure ...,Jason Headley Matthew Aldrich Chris Evans Keke...
4,Spiderhead,not-released,5.4,23K,"[Action, Crime, Drama]","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/,discover medical test reality fictional drug v...,Rhett Reese Paul Wernick Chris Hemsworth Miles...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24397,Delicatessen,"FRF 24,000,000 (estimated)",7.6,85K,"[Comedy, Crime]",Post-apocalyptic surrealist black comedy about...,"['surrealist', 'black comedy', 'human meat', '...",Marc Caro,"['Jean-Pierre Jeunet', 'Marc Caro', 'Gilles Ad...",Jean-Pierre Jeunet,-1991,/title/tt0101700/,surrealist black comedy human meat absurd come...,Jean-Pierre Jeunet Marc Caro Gilles Adrien Mar...
24398,Bitch Ass,not-released,5.5,52,"[Crime, Horror]",A gang initiation goes wrong when a group of f...,[],Bill Posley,"['Bill Posley', 'Teon Kelley', 'Tunde Laleye',...",Jonathan Colomb,-2022,/title/tt13991504/,,Bill Posley Teon Kelley Tunde Laleye Me'lisa S...
24399,Bullwhip,not-released,5.1,398,"[Crime, Romance, Western]","In order to avoid the hangman's noose, a cowbo...","['taming of the shrew', 'fur trader', 'busines...",Harmon Jones,"['Guy Madison', 'Rhonda Fleming', 'James Griff...",Adele Buffington,-1958,/title/tt0051438/,taming of the shrew fur trader business rival ...,Guy Madison Rhonda Fleming James Griffith Harm...
24400,The Freshman,1 hour 42 minutes,6.4,20K,"[Comedy, Crime]",An N.Y.C. film school student accepts a job wi...,"['endangered species', 'fish out of water', 'g...",Andrew Bergman,"['Marlon Brando', 'Matthew Broderick', 'Bruno ...",Andrew Bergman,-1990,/title/tt0099615/,endangered species fish out of water gangster ...,Marlon Brando Matthew Broderick Bruno Kirby An...


In [36]:
unique_generes = df['Generes'].explode().unique()
unique_generes

array(['Action', 'Drama', 'Adventure', 'Sci-Fi', 'Animation', 'Crime',
       'Comedy', 'Thriller', 'Fantasy', 'Horror', 'History', 'Mystery',
       'Biography', 'War', 'Western', 'Sport', 'Family', 'Romance',
       'Music', 'Musical', 'Film-Noir', 'Game-Show', 'Adult',
       'Reality-TV'], dtype=object)

In [37]:
df.drop(['Plot Kyeword','Top 5 Casts'],axis=1, inplace=True)

In [39]:
df['text'] = df.apply(lambda x : str(x['Overview'])+' '+x['Keywords']+' '+x['Stars'], axis=1)
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...,As students at the United States Navy's elite ...


In [169]:
# Function to parse vote count into an integer
def parse_vote_count(vote_count_str):
    if isinstance(vote_count_str, str):
        vote_count_str = vote_count_str.replace('K', '000').replace('M', '000000').replace('.', '')
    return int(vote_count_str)

# Apply the function to the Vote count column
df['User Rating'] = df['User Rating'].apply(parse_vote_count)

/var/folders/q9/t4wc2v252nx85zmpldx4t9_00000gn/T/ipykernel_18600/1234330232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User Rating'] = df['User Rating'].apply(parse_vote_count)


In [ ]:
df = df[df['Overview'].str.strip()!='']
df

In [ ]:
df['ids'] = df.index
df['ids'] = df['ids'].astype('str')
# Clean up 'year' column by removing the leading '-'
df['year'] = df['year'].fillna("0").str.extract('(\d+)').fillna(0).astype(int)

In [45]:
model = SentenceTransformer('intfloat/multilingual-e5-large')

In [49]:
# Get embeddings for the 'text' column
embeddings = df['text'].apply(lambda x: model.encode(x, batch_size=64, show_progress_bar=True))
df['embeddings'] = embeddings.tolist()
df[['text', 'embeddings']].head()

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.09it/s]


,text,embeddings
0,After more than thirty years of service as one...,"[0.030983549, -0.014697521, -0.027656412, -0.0..."
1,Four years after the destruction of Isla Nubla...,"[0.013468823, -0.0013912105, -0.013709203, -0...."
2,As students at the United States Navy's elite ...,"[0.004364043, 0.013403226, -0.011268646, -0.05..."
3,While spending years attempting to return home...,"[0.024983224, 0.011186211, 7.8756704e-05, -0.0..."
4,"In the near future, convicts are offered the c...","[0.0033428825, -0.010396928, -0.015175045, -0...."


In [154]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1024,
              'host': 'movies-embeddings-kno6onc.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'movies-embeddings',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [170]:

dimensions_embeddings = len(df['embeddings'][0])
index_name = 'movies-embeddings'

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)
pc = Pinecone(api_key=pinecone_api_key, environment = spec)


In [189]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1024,
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
else:
    print('index already created')
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [190]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text',],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb,metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

100%|██████████| 378/378 [06:13<00:00,  1.01it/s]


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 24576}},
 'total_vector_count': 24576}

In [124]:

def query(payload):
	API_URL = "https://api-inference.huggingface.co/models/intfloat/multilingual-e5-large"
	headers = {"Authorization": "Bearer " + api_hugging,
						"x-wait-for-model": "true"}

	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

query_str = "space travel movie"
query_vector = query({
	"inputs": query_str,
})

In [120]:
output

[0.008589547127485275,
 0.003369762795045972,
 -0.03191767632961273,
 -0.04185143858194351,
 0.02470574341714382,
 -0.014080403372645378,
 0.008880638517439365,
 0.04344075918197632,
 0.032754454761743546,
 -0.032436124980449677,
 0.038970980793237686,
 -0.00696781650185585,
 -0.04016082361340523,
 -0.023932427167892456,
 -0.036349982023239136,
 -0.007614491041749716,
 0.016644448041915894,
 0.012335507199168205,
 0.027043459936976433,
 0.004316329024732113,
 0.05435600131750107,
 -0.04261740669608116,
 -0.032864052802324295,
 -0.02863847091794014,
 -0.012429369613528252,
 -0.02860419824719429,
 -0.023779544979333878,
 -0.020059270784258842,
 -0.02592582069337368,
 -0.05141888186335564,
 -0.022199345752596855,
 -0.00043736607767641544,
 -0.035308707505464554,
 -0.014959965832531452,
 -0.012909234501421452,
 0.0401868000626564,
 0.06430396437644958,
 0.037558913230895996,
 -0.05759360268712044,
 0.009769407100975513,
 -0.03418026864528656,
 0.02887088805437088,
 -0.0052791922353208065,


In [192]:
def search(query_str, genre, rating, year, voting_count, top_k):
    query_vector = query({
        "inputs": query_str,
    })

    if rating:
        filter_rating = rating
    else:
        filter_rating = 0

    if year:
        filter_year = year
    else:
        filter_year = 0

    if voting_count:
        filter_voting_count = voting_count
    else:
        filter_voting_count = 0

    if genre:
         conditions ={
                "Generes": { "$in": [genre] },
                "Rating": { "$gte": filter_rating },
                "year": { "$gte": filter_year },
                "User Rating": { "$gte": filter_voting_count }
                }
    else:
        conditions ={
                "Rating": { "$gte": filter_rating },
                "year": { "$gte": filter_year },
                "User Rating": { "$gte": filter_voting_count }
                }

    responses = index.query(
        vector=query_vector,
        top_k=top_k,
        include_metadata=True,
        filter=conditions
    )

    # Format the responses for better display
    response_data = []
    for response in responses['matches']:
        response_data.append({
            'Title': response['metadata']['movie title'],
            'Overview': response['metadata']['Overview'],
            'Director': response['metadata']['Director'],
            'Genre': response['metadata']['Generes'],
            'year': int(response['metadata']['year']),
            'Rating': response['metadata']['Rating'],
            'Stars': response['metadata']['Stars'],
            #'Vote count': response['metadata']['User Rating'],
            #'Score': response['score'],
            'Link': response['metadata']['path'],
        })

    df = pd.DataFrame(response_data)
    return df

search("space travel movie", "Action", 6, 1970, 200, 10)


,Title,Overview,Director,Genre,year,Rating,Stars,Link
0,Junk Head,It is a real SF feature film used in stop moti...,Takahide Hori,"[Animation, Action, Adventure]",2017,7.5,Takahide Hori Hamdullah Saat Takahide Hori Tak...,/title/tt6848928/
1,Buck Rogers in the 25th Century,A 20th century astronaut emerges out of 500 ye...,Daniel Haller,"[Action, Adventure, Sci-Fi]",1979,6.5,Leslie Stevens Philip Francis Nowlan Gil Gerar...,/title/tt0077278/
2,Koi... Mil Gaya,A developmentally disabled young man tries to ...,Rakesh Roshan,"[Action, Drama, Romance]",2003,7.1,Honey Irani Sachin Bhowmick Hrithik Roshan Rek...,/title/tt0254481/
3,Casshern,Live-action sci-fi movie based on a 1973 Japan...,Kazuaki Kiriya,"[Action, Sci-Fi]",2004,6.0,Dai Satô Shotaro Suga Yûsuke Iseya Kumiko Asô ...,/title/tt0405821/
4,Proxima,An astronaut prepares for a one-year mission a...,Alice Winocour,"[Action, Adventure, Drama]",2019,6.3,Jean-Stéphane Bron Eva Green Zélie Boulant Mat...,/title/tt7374926/
5,Lockout,A man wrongly convicted of conspiracy to commi...,James Mather,"[Action, Sci-Fi, Thriller]",2012,6.0,James Mather Steve Saint Leger Luc Besson Guy ...,/title/tt1592525/
6,Independence Day,The aliens are coming and their goal is to inv...,Roland Emmerich,"[Action, Adventure, Sci-Fi]",1996,7.0,Roland Emmerich Will Smith Bill Pullman Jeff G...,/title/tt0116629/
7,The Fifth Element,"In the colorful future, a cab driver unwitting...",Luc Besson,"[Action, Adventure, Sci-Fi]",1997,7.6,Robert Mark Kamen Bruce Willis Milla Jovovich ...,/title/tt0119116/
8,Stargate,"An interstellar teleportation device, found in...",Roland Emmerich,"[Action, Adventure, Sci-Fi]",1994,7.0,Roland Emmerich Kurt Russell James Spader Jaye...,/title/tt0111282/
9,Starship Troopers,"Humans in a fascist, militaristic future wage ...",Paul Verhoeven,"[Action, Adventure, Sci-Fi]",1997,7.3,Robert A. Heinlein Casper Van Dien Denise Rich...,/title/tt0120201/


In [180]:
search('viaje en el tiempo', "Action", 6, 1970, 200, 10)


,Title,Overview,Director,Genre,year,Rating,Vote count
0,Terminator 3: Rise of the Machines,A machine from a post-apocalyptic future trave...,Jonathan Mostow,"[Action, Sci-Fi]",2003,6.3,395000.0
1,24,"A scientist invents a time machine, which lead...",Vikram K. Kumar,"[Action, Comedy, Drama]",2016,7.9,22000.0
2,Men in Black 3,Agent J travels in time to M.I.B.'s early days...,Barry Sonnenfeld,"[Action, Adventure, Comedy]",2012,6.8,358000.0
3,Yamudiki Mogudu,"A man, who wrongfully dies before his time, is...",Ravi Raja Pinisetty,"[Action, Comedy, Fantasy]",1988,7.3,327.0
4,Doctor Strange,While on a journey of physical and spiritual h...,Scott Derrickson,"[Action, Adventure, Fantasy]",2016,7.5,715000.0
5,Contratiempo,A race against time between a man whose son re...,Ramiro Medina Flores,[Action],2011,7.7,17000.0
6,Paycheck,What seemed like a breezy idea for an engineer...,John Woo,"[Action, Mystery, Sci-Fi]",2003,6.3,108000.0
7,The Final Countdown,A modern aircraft carrier is thrown back in ti...,Don Taylor,"[Action, Adventure, Sci-Fi]",1980,6.6,24000.0
8,The Terminator,A human soldier is sent from 2029 to 1984 to s...,James Cameron,"[Action, Sci-Fi]",1984,8.1,846000.0
9,Koi... Mil Gaya,A developmentally disabled young man tries to ...,Rakesh Roshan,"[Action, Drama, Romance]",2003,7.1,24000.0


In [179]:
search('comedia romantica con un perro', "Action", 6, 1970, 200, 10)


,Title,Overview,Director,Genre,year,Rating,Vote count
0,K-9,"To stop an elusive criminal, a maverick detect...",Rod Daniel,"[Action, Comedy, Crime]",1989,6.0,34000.0
1,Keanu,When the adorable kitten of an L.A. crime king...,Peter Atencio,"[Action, Comedy, Crime]",2016,6.2,49000.0
2,Next Gen,A friendship with a top-secret robot turns a l...,Kevin R. Adams,"[Animation, Action, Adventure]",2018,6.6,17000.0
3,Unleashed,A man enslaved by the mob since childhood and ...,Louis Leterrier,"[Action, Crime, Drama]",2005,7.0,103000.0
4,The Doberman Gang,A clever con artist and an animal trainer team...,Byron Chudnow,"[Action, Comedy, Crime]",1972,6.0,11000.0
5,Violet & Daisy,Two teenage assassins accept what they think w...,Geoffrey Fletcher,"[Action, Comedy, Crime]",2011,6.0,13000.0
6,Uno sceriffo extraterrestre... poco extra e mo...,A young humanoid alien who gets stranded on ea...,Michele Lupo,"[Action, Comedy, Family]",1979,6.1,38000.0
7,Mr. And Mrs. Ramchari,A man whose life seems straight out of a class...,Santhosh Ananddram,"[Action, Drama, Romance]",2014,7.3,23000.0
8,Cabras da Peste,Two hapless cops find themselves in over their...,Vitor Brandt,"[Action, Comedy, Crime]",2021,6.0,18000.0
9,Avalon,"In a dystopian world, a woman spends her time ...",Mamoru Oshii,"[Action, Drama, Fantasy]",2001,6.4,13000.0


In [123]:
df

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text,embeddings,ids
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...,"[0.030983549, -0.014697521, -0.027656412, -0.0...",0
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...,"[0.013468823, -0.0013912105, -0.013709203, -0....",1
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...,As students at the United States Navy's elite ...,"[0.004364043, 0.013403226, -0.011268646, -0.05...",2
3,Lightyear,"$71,101,257",5.2,32K,"[Animation, Action, Adventure]",While spending years attempting to return home...,Angus MacLane,Angus MacLane,2022,/title/tt10298810/,galaxy spaceship robot rocket space adventure ...,Jason Headley Matthew Aldrich Chris Evans Keke...,While spending years attempting to return home...,"[0.024983224, 0.011186211, 7.8756704e-05, -0.0...",3
4,Spiderhead,not-released,5.4,23K,"[Action, Crime, Drama]","In the near future, convicts are offered the c...",Joseph Kosinski,George Saunders,2022,/title/tt9783600/,discover medical test reality fictional drug v...,Rhett Reese Paul Wernick Chris Hemsworth Miles...,"In the near future, convicts are offered the c...","[0.0033428825, -0.010396928, -0.015175045, -0....",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24397,Delicatessen,"FRF 24,000,000 (estimated)",7.6,85K,"[Comedy, Crime]",Post-apocalyptic surrealist black comedy about...,Marc Caro,Jean-Pierre Jeunet,1991,/title/tt0101700/,surrealist black comedy human meat absurd come...,Jean-Pierre Jeunet Marc Caro Gilles Adrien Mar...,Post-apocalyptic surrealist black comedy about...,"[0.0058069797, -0.0012189006, -0.0020962523, -...",24397
24398,Bitch Ass,not-released,5.5,52,"[Crime, Horror]",A gang initiation goes wrong when a group of f...,Bill Posley,Jonathan Colomb,2022,/title/tt13991504/,,Bill Posley Teon Kelley Tunde Laleye Me'lisa S...,A gang initiation goes wrong when a group of f...,"[0.039626524, -0.008098167, -0.049389143, -0.0...",24398
24399,Bullwhip,not-released,5.1,398,"[Crime, Romance, Western]","In order to avoid the hangman's noose, a cowbo...",Harmon Jones,Adele Buffington,1958,/title/tt0051438/,taming of the shrew fur trader business rival ...,Guy Madison Rhonda Fleming James Griffith Harm...,"In order to avoid the hangman's noose, a cowbo...","[0.03266498, -0.0068279086, -0.022606296, -0.0...",24399
24400,The Freshman,1 hour 42 minutes,6.4,20K,"[Comedy, Crime]",An N.Y.C. film school student accepts a job wi...,Andrew Bergman,Andrew Bergman,1990,/title/tt0099615/,endangered species fish out of water gangster ...,Marlon Brando Matthew Broderick Bruno Kirby An...,An N.Y.C. film school student accepts a job wi...,"[0.010003991, -0.009481709, -0.032525685, -0.0...",24400
